# About

Join 2022-11-05
- This time, get the three nearest pollution sources per school. 

Pretty much do everything in pandas until the very end, do the cross join etc. in spark to make it parallelized.

In [2]:
import pandas as pd 
import numpy as np
import os 
import datetime
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as mticker
import plotly.express as px

# spark
from pyspark.sql import SparkSession
from pyspark.sql import types, functions as F, Window

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [3]:
# local or gdrive
path_source = 'work'

if path_source == 'gdrive':
  from google.colab import drive
  drive.mount('/content/gdrive')
  data_path = '/content/gdrive/MyDrive/Classes/W210_capstone/W210_Capstone/Data'
  
elif path_source == 'local':
  data_path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210_Capstone/Data'

elif path_source == 'work':
  data_path = '/Users/trevorjohnson/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210_Capstone/Data'

In [4]:
df_census = pd.read_csv(os.path.join(data_path, 'census/census_bureau_clean/census_bureau.csv'))
df_wind = pd.read_parquet(os.path.join(data_path, 'wind'))
df_pollution = pd.read_csv(os.path.join(data_path, 'AirPollution/UW_pm25_zip_monthly_anand_2000-2018-v2.csv'))
df_point_sources = pd.read_csv(os.path.join(data_path, 'Point source/pollution_point_sources.csv'))
# df_schools = pd.read_csv(os.path.join(data_path, 'schools/filtered_joined_schools_data.csv'))

file_encoding = 'utf8'
with open(os.path.join(data_path, 'schools/filtered_joined_schools_data.csv'), encoding=file_encoding, errors = 'backslashreplace') as my_csv:
  df_schools = pd.read_csv(my_csv, low_memory=False)

In [5]:
# clean schools 
df_schools.columns = [i.lower() for i in df_schools.columns]
# only select necessary fields
df_schools = df_schools[['cdscode', 'statustype', 'county', 'street', 'zip_first_five', 'opendate', 'closeddate', 'eilname', 'gsoffered', 
  'latitude', 'longitude', 'lastupdate']]\
  .rename(columns={
    'statustype': 'school_active_status', 'county': 'school_county', 'street': 'school_street', 
    'zip_first_five': 'school_zip', 'opendate': 'school_open_date', 'closeddate': 'school_closed_date', 
    'eilname': 'school_type', 'gsoffered': 'school_grades_offered', 'latitude': 'school_lat', 'longitude': 'school_lon', 
    'lastupdate': 'school_last_updated_date'})

# clean wind
df_wind = df_wind.rename(columns={'lat': 'wind_lat', 'lon': 'wind_lon'})
df_wind['year_month'] = df_wind['year_month'].astype(str).map(lambda x: x[:4] + '-' + x[-2:])
df_wind['year'] = df_wind['year_month'].map(lambda x: int(x[:4]))
df_wind['ZCTA10'] = df_wind['ZCTA10'].astype(int)
df_wind = df_wind[(df_wind['year'] >= 2000) & (df_wind['year'] <= 2019)]

# clean pollution
df_pollution = df_pollution.drop(columns=['Unnamed: 0', 'GEOID10', 'year_month_zip'])

# clean pollution point sources
df_point_sources = df_point_sources.rename(columns={'zip_code': 'point_source_zip'})
df_point_sources['point_source_zip'] = df_point_sources['point_source_zip'].astype(int)
# create an ID field for easier lookups
df_point_sources['point_source_id'] = [i for i in range(df_point_sources.shape[0])]

In [6]:
df_all = pd.merge(df_schools, df_wind, left_on = 'school_zip', right_on='ZCTA10', how='left')\
  .merge(df_census, left_on = ['school_zip', 'year'], right_on=['zip', 'year'], how='left')\
  .merge(df_pollution, left_on=['school_zip', 'year_month'], right_on=['ZIP10', 'year_month'], how='left')

In [7]:
# QA Checks
# each school is repeated for every year-month combo. But some schools dont have wind/population data where we dont have that zip code in those datasets. 
yr_mo = df_wind[['year_month']].drop_duplicates().shape[0]
print(f'There are {yr_mo} year month combos')
print('So most schools are repeated 240 times, for the schools that dont have a zip code in the wind data, there are no obs')
df_all['cdscode'].value_counts().to_frame().value_counts('cdscode')

There are 240 year month combos
So most schools are repeated 240 times, for the schools that dont have a zip code in the wind data, there are no obs


cdscode
240    12426
1        871
dtype: int64

## Lat/Lon Join

Join the above dataset to pull the nearest pollution source by year. 

Do so by creating a school <--> source mapping by year

In [8]:
# here is the year mapping since we dont have all years available in the pollution sources. 
# thus, we have to interpolate for the missing years
year_mapping = {
  2000: 2002, 
  2001: 2002, 
  2002: 2002,
  2003: 2005,
  2004: 2005,
  2005: 2005,
  2006: 2008,
  2007: 2008,
  2008: 2008,
  2009: 2011,
  2010: 2011,
  2011: 2011,
  2012: 2014,
  2013: 2014,
  2014: 2014,
  2015: 2017,
  2016: 2017,
  2017: 2017,
  2018: 2017,
  2019: 2017
}

In [9]:
spark = SparkSession\
    .builder\
    .appName('')\
    .master("local[*]")\
    .getOrCreate()

sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/05 15:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
df_schools = df_all[['cdscode', 'year', 'school_lat', 'school_lon']].drop_duplicates()
df_ps = df_point_sources[['point_source_id', 'checked_lat', 'checked_lon', 'point_source_zip', 'report_year', 'PM25_emissions_TPY']]

In [11]:
def get_top_n_polluters(data_year, topn = 3, verbose=True):
  
  if verbose:
    print('Working on year: {}'.format(data_year))

  # get spark df of individual year
  df_ps_i = spark.createDataFrame(df_ps[df_ps.report_year == year_mapping[data_year]])
  df_school_yr_i = spark.createDataFrame(df_schools[df_schools.year == data_year])

  # cross join, calc distance, then rank the distances, then filter on top 3
  df_x = df_school_yr_i.crossJoin(df_ps_i)\
    .withColumn('distance_euclid', ((F.col('school_lat') - F.col('checked_lat'))**2 + (F.col('school_lon') - F.col('checked_lon'))**2)**.5)\
    .withColumn('dist_rank', F.row_number().over(Window().partitionBy('cdscode').orderBy(F.col('distance_euclid'))))\
    .filter(F.col('dist_rank') <= topn)
  
  return df_x.toPandas()

In [12]:
# run the function on every year!
top_polluters_df = pd.concat([get_top_n_polluters(data_year, topn=5) for data_year in range(2000, 2020)])

Working on year: 2000


Working on year: 2001


Working on year: 2002


Working on year: 2003


Working on year: 2004


...

Working on year: 2015


Working on year: 2016


Working on year: 2017


Working on year: 2018


Working on year: 2019


In [13]:
# every school/year combo has n records
top_polluters_df.value_counts(['cdscode', 'year']).to_frame().reset_index().rename(columns={0: 'n'}).value_counts('n')

n
5    248520
dtype: int64

In [14]:
top_polluters_df.head(3)

,cdscode,year,school_lat,school_lon,point_source_id,checked_lat,checked_lon,point_source_zip,report_year,PM25_emissions_TPY,distance_euclid,dist_rank
0,1611190131805,2000.0,37.77693,-122.28528,518,37.789864,-122.293587,94501,2002,6.915300,0.015372,1
1,1611190131805,2000.0,37.77693,-122.28528,163,37.798897,-122.282364,94607,2002,2.631207,0.022160,2
2,1611190131805,2000.0,37.77693,-122.28528,248,37.790672,-122.261948,94606,2002,3.168564,0.027078,3


In [15]:
all_schools = list(set(df_schools['cdscode'].values))
res = [i not in top_polluters_df['cdscode'].values for i in all_schools]
missing_schools = np.array(all_schools)[res]
print('{} schools missing from the final lookup.\nThese are the schools that dont have a valid year field'.format(len(missing_schools)))
df_schools[df_schools['cdscode'].isin(missing_schools)].shape[0]

871 schools missing from the final lookup.
These are the schools that dont have a valid year field


871

In [18]:
print('In the end, we should end up with this many records.\nB/c there are 3 nearest polluters for each school for each year')
print(df_schools[['cdscode', 'year']].drop_duplicates().shape[0] * 3)

# the
res_count = top_polluters_df.shape[0] + 871*3
print('Here is the count in the end: {}. We have to add in the 871 missing schools that have NA year'.format(res_count))

In the end, we should end up with this many records.
B/c there are 3 nearest polluters for each school for each year
748173
Here is the count in the end: 1245213. We have to add in the 871 missing schools that have NA year


In [19]:
top_polluters_df = top_polluters_df.rename(columns = {'checked_lat': 'ps_lat', 'checked_lon': 'ps_lon'})

In [21]:
top_polluters_df.head(10)

,cdscode,year,school_lat,school_lon,point_source_id,ps_lat,ps_lon,point_source_zip,report_year,PM25_emissions_TPY,distance_euclid,dist_rank
0,1611190131805,2000.0,37.776930,-122.28528,518,37.789864,-122.293587,94501,2002,6.915300,0.015372,1
1,1611190131805,2000.0,37.776930,-122.28528,163,37.798897,-122.282364,94607,2002,2.631207,0.022160,2
2,1611190131805,2000.0,37.776930,-122.28528,248,37.790672,-122.261948,94606,2002,3.168564,0.027078,3
3,1611190131805,2000.0,37.776930,-122.28528,428,37.819660,-122.285744,94607,2002,5.147391,0.042733,4
4,1611190131805,2000.0,37.776930,-122.28528,66,37.773750,-122.237137,94606,2002,2.102984,0.048248,5
5,1611190139220,2000.0,37.772663,-122.27145,248,37.790672,-122.261948,94606,2002,3.168564,0.020362,1
6,1611190139220,2000.0,37.772663,-122.27145,518,37.789864,-122.293587,94501,2002,6.915300,0.028034,2
7,1611190139220,2000.0,37.772663,-122.27145,163,37.798897,-122.282364,94607,2002,2.631207,0.028414,3
8,1611190139220,2000.0,37.772663,-122.27145,66,37.773750,-122.237137,94606,2002,2.102984,0.034330,4
9,1611190139220,2000.0,37.772663,-122.27145,677,37.772911,-122.234818,94606,2002,13.356216,0.036633,5


In [22]:
top_polluters_df2 = top_polluters_df.pivot(
  index = ['cdscode', 'year', 'school_lat', 'school_lon'], 
  columns = ['dist_rank'], 
  values = ['point_source_id', 'ps_lat', 'ps_lon', 'PM25_emissions_TPY', 'distance_euclid']
  )

top_polluters_df2.head(3)

point_source_id                \
dist_rank                                                1      2      3   
cdscode       year   school_lat school_lon                                 
1100170000000 2000.0 37.658212  -122.09713           774.0  608.0  385.0   
              2001.0 37.658212  -122.09713           774.0  608.0  385.0   
              2002.0 37.658212  -122.09713           774.0  608.0  385.0   

                                                             ps_lat  \
dist_rank                                       4      5          1   
cdscode       year   school_lat school_lon                            
1100170000000 2000.0 37.658212  -122.09713  661.0  140.0  37.667973   
              2001.0 37.658212  -122.09713  661.0  140.0  37.667973   
              2002.0 37.658212  -122.09713  661.0  140.0  37.667973   

                                                                             \
dist_rank                                           2          3          4   
cdscode       year   school_lat school_lon                                    
1100170000000 2000.0 37.658212  -122.09713  37.633793  37.635574  37.701672   
              2001.0 37.658212  -122.09713  37.633793  37.635574  37.701672   
              2002.0 37.658212  -122.09713  37.633793  37.635574  37.701672   

                                                           ps_lon              \
dist_rank                                           5           1           2   
cdscode       year   school_lat school_lon                                      
1100170000000 2000.0 37.658212  -122.09713  37.617073 -122.103806 -122.124954   
              2001.0 37.658212  -122.09713  37.617073 -122.103806 -122.124954   
              2002.0 37.658212  -122.09713  37.617073 -122.103806 -122.124954   

                                                                    \
dist_rank                                            3           4   
cdscode       year   school_lat school_lon                           
1100170000000 2000.0 37.658212  -122.09713 -122.052284 -122.138557   
              2001.0 37.658212  -122.09713 -122.052284 -122.138557   
              2002.0 37.658212  -122.09713 -122.052284 -122.138557   

                                                       PM25_emissions_TPY  \
dist_rank                                            5                  1   
cdscode       year   school_lat school_lon                                  
1100170000000 2000.0 37.658212  -122.09713 -122.047699          25.813472   
              2001.0 37.658212  -122.09713 -122.047699          25.813472   
              2002.0 37.658212  -122.09713 -122.047699          25.813472   

                                                                           \
dist_rank                                          2         3          4   
cdscode       year   school_lat school_lon                                  
1100170000000 2000.0 37.658212  -122.09713  9.581375  4.616791  11.998904   
              2001.0 37.658212  -122.09713  9.581375  4.616791  11.998904   
              2002.0 37.658212  -122.09713  9.581375  4.616791  11.998904   

                                                  distance_euclid           \
dist_rank                                       5               1        2   
cdscode       year   school_lat school_lon                                   
1100170000000 2000.0 37.658212  -122.09713  2.492        0.011826  0.03702   
              2001.0 37.658212  -122.09713  2.492        0.011826  0.03702   
              2002.0 37.658212  -122.09713  2.492        0.011826  0.03702   

                                                                          
dist_rank                                          3         4         5  
cdscode       year   school_lat school_lon                                
1100170000000 2000.0 37.658212  -122.09713  0.050236  0.060041  0.064311  
              2001.0 37.658212  -122.09713  0.050236  0.060041  0.064311  
      

In [23]:
top_polluters_df2.shape[0]

248520

In [ ]:
# write this mapping table as parquet to disk
# school_ps_mapping_df = pd.concat(school_ps_mapping)
# fpath = os.path.join(data_path, 'school_pollution_mapping/school_pollution_mapping_top3.parquet')
# school_ps_mapping_df.to_parquet(fpath)

In [ ]:
# finally, cast the rows out wider for the mapping. 